# U-Net Brain Tumor Segmentation

In [2]:
import tensorflow as tf
import keras
from keras.callbacks import Callback
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import TensorBoard

import numpy as np
import os
import SimpleITK as sitka
import random
import matplotlib.pyplot as plt
import importlib
import time

# Data augmentation
import cv2
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, ElasticTransform, Rotate, RandomScale, GridDistortion, OpticalDistortion, OneOf
)

# Custom scripts
import datagen
import model as unet_model_script

# Reload lib contents
importlib.reload(datagen)
importlib.reload(unet_model_script)

<module 'model' from 'C:\\Users\\krcma\\Desktop\\tf_segmentation\\unet_segmentation\\model.py'>

##  Hyperparemeters

In [3]:
train_HGG_patients = 239
valid_HGG_patients = 20
slices_from_patient = 15

image_size = 240
channels = 4
learning_rate = 1e-4
epochs = 25
batch_size = 7


### U-Net model parameters

Configuration dict.

In [4]:
conf = {
    "last_layer_activation": "sigmoid",
    "dropout": 0.05,
    "batch_norm": True
}

In [5]:
training_model_conf = {
    "optimizer": keras.optimizers.Adam(learning_rate=learning_rate),
    "loss": "binary_crossentropy",
    "metrics": ["accuracy"]
}

## Dataset

In [6]:
x_train, y_train = datagen.get_dataset(slices_from_patient, 
                                        file_path='../dataset/', 
                                        mode='training', 
                                        glioma_type=['HGG'], 
                                        slices_w_tumor_only=100,
                                        slices_w_less_brain=None,
                                        image_size=image_size,
                                        train_HGG_patients=train_HGG_patients)

training data with shape: (3585, 4, 240, 240) (3585, 1, 240, 240)


In [7]:
x_valid, y_valid = datagen.get_dataset(slices_from_patient, 
                                        file_path='../dataset/', 
                                        mode='validation', 
                                        glioma_type=['HGG'], 
                                        slices_w_tumor_only=False,
                                        slices_w_less_brain=None,
                                        image_size=240,
                                        train_HGG_patients=valid_HGG_patients)

validation data with shape: (300, 4, 240, 240) (300, 1, 240, 240)


## Data augmentation

In [8]:
AUGMENTATIONS = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
#     Rotate(limit=90, p=0.99),
#     RandomGamma(gamma_limit=(80, 120), p=0.25),
#     RandomScale(scale_limit=0.1, interpolation=cv2.INTER_NEAREST, p=0.25),
#     ElasticTransform(alpha=1, p=0.5),
#     OneOf([
#         OpticalDistortion(p=0.3),
#         GridDistortion(p=0.1),
#     ], p=0.2),
    OpticalDistortion(p=0.5),
    GridDistortion(p=0.5),
])

## Callbacks

In [9]:
class NegativeLoss(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        if logs.get('loss') < 0:
            print("\nEarly stopping for negative loss ({}) at batch {}.".format(logs.get('loss'), batch))
            self.model.stop_training = True

callbacks = [
    NegativeLoss(),
    keras.callbacks.callbacks.TerminateOnNaN(),
    keras.callbacks.callbacks.EarlyStopping(monitor='accuracy', 
                                            min_delta=0.0001, 
                                            patience=10, 
                                            verbose=1, 
                                            mode='auto', 
                                            baseline=None, 
                                            restore_best_weights=True),
    keras.callbacks.callbacks.ReduceLROnPlateau(factor=0.1, 
                                                patience=5, 
                                                min_lr=0.00001, 
                                                verbose=1),
#     keras.callbacks.TensorBoard(log_dir="../logs/{}".format(time.time()))
]

# Ensemble training

In [ ]:
models = {}
for tumor_region in range(1, 2):
    
    print("Training U-Net {}".format(tumor_region))
    
    # Get and compile model
    models[tumor_region] = unet_model_script.UNet(channels, image_size, conf=conf)
    models[tumor_region].compile(**training_model_conf)
    
    # Convert mask labels to binary
    train_masks = datagen.convert_labels(y_train, tumor_region)
    valid_masks = datagen.convert_labels(y_valid, tumor_region)
    
    # Init datagen w augmentation
    train_gen = datagen.AugmentationDatagen(x_train, train_masks, batch_size, AUGMENTATIONS)
    
    # Start training
    time_start = time.time()
    models["hist_{}".format(tumor_region)] = models[tumor_region].fit(train_gen, 
                                                                      validation_data=(x_valid, valid_masks), 
                                                                      epochs=epochs, 
                                                                      shuffle=True, 
                                                                      callbacks=callbacks)
    print("Training time: {}".format(time.time()-time_start))
    
#     model.save("../models/ensemble_{}.h5".format(time.time(), tumor_region))
    

Training U-Net 1
Epoch 1/25
513/513 [==============================] - 279s 544ms/step - loss: 0.0956 - accuracy: 0.9887 - val_loss: 0.0570 - val_accuracy: 0.9902
Epoch 2/25
513/513 [==============================] - 261s 509ms/step - loss: 0.0605 - accuracy: 0.9874 - val_loss: 0.0459 - val_accuracy: 0.9890
Epoch 3/25
513/513 [==============================] - 269s 525ms/step - loss: 0.0429 - accuracy: 0.9889 - val_loss: 0.1283 - val_accuracy: 0.9861
Epoch 4/25
513/513 [==============================] - 317s 619ms/step - loss: 0.0360 - accuracy: 0.9894 - val_loss: 0.0331 - val_accuracy: 0.9908
Epoch 5/25
513/513 [==============================] - 310s 604ms/step - loss: 0.0309 - accuracy: 0.9901 - val_loss: 0.0306 - val_accuracy: 0.9900
Epoch 6/25
285/513 [===============>..............] - ETA: 2:18 - loss: 0.0281 - accuracy: 0.9906

### Results

In [ ]:
tumor_region = 1

# Plot training & validation accuracy values
plt.plot(models["hist_{}".format(tumor_region)].history['accuracy'])
plt.plot(models["hist_{}".format(tumor_region)].history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(models["hist_{}".format(tumor_region)].history['loss'])
plt.plot(models["hist_{}".format(tumor_region)].history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(x_valid, verbose=1)